<a href="https://colab.research.google.com/github/ars200200/machine_learning/blob/main/catsdogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision as tv

import numpy as np
import matplotlib.pyplot as plt
import cv2


import tqdm
import os


In [ ]:
%matplotlib
%matplotlib inline

Using matplotlib backend: QtAgg


In [ ]:
class Dataset2class(torch.utils.data.Dataset):
    def __init__(self, path_dir1:str, path_dir2:str):
        super().__init__()

        self.path_dir1 = path_dir1
        self.path_dir2 = path_dir2

        self.dir1_list = sorted(os.listdir(path_dir1))
        self.dir2_list = sorted(os.listdir(path_dir2))

    def __len__(self):
        return len(self.dir2_list) + len(self.dir1_list)

    def __getitem__(self, idx :int):

        if idx < len(self.dir1_list):
            class_id = 0
            img_path = os.path.join(self.path_dir1, self.dir1_list[idx])
        else:
            class_id = 1
            idx -= len(self.dir1_list)
            img_path = os.path.join(self.path_dir2, self.dir2_list[idx])

        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img /= 255.0

        img = cv2.resize(img, (64, 64),  interpolation = cv2.INTER_AREA)
        img = img.transpose((2, 0, 1))

        t_img = torch.from_numpy(img)
        t_class_id = torch.tensor(class_id)

        return {'img': t_img, 'label': t_class_id}


os.listdir()

['.ipynb_checkpoints', 'archive.zip', 'catsdogs.ipynb', 'Cv2.ipynb', 'dataset']

In [ ]:
train_ds_catsdogs = Dataset2class(
    './dataset/training_set/dogs',
    './dataset/training_set/cats'
)

test_ds_catsdogs = Dataset2class(
    './dataset/test_set/dogs',
    './dataset/test_set/cats'
)

In [ ]:
len(train_ds_catsdogs)

8000

In [ ]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(
    train_ds_catsdogs, shuffle=True,
    batch_size=batch_size, num_workers=0, drop_last=True

)
test_loader = torch.utils.data.DataLoader(
    test_ds_catsdogs, shuffle=True,
    batch_size=batch_size, num_workers=0


)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.act = nn.LeakyReLU(0.2)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.conv0  = nn.Conv2d(3, 32, 3, stride=1, padding=0)
        self.conv1  = nn.Conv2d(32, 32, 3, stride=1, padding=0)
        self.conv2  = nn.Conv2d(32, 64, 3, stride=1, padding=0)
        self.conv3  = nn.Conv2d(64, 128, 3, stride=1, padding=0)

        self.adaptivepool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()

        self.linear = nn.Linear(128, 20)
        self.linear2 = nn.Linear(20, 2)

    def forward(self, x):

        out = self.conv0(x)
        out = self.act(out)
        out = self.maxpool(out)

        out = self.conv1(out)
        out = self.act(out)
        out = self.maxpool(out)

        out = self.conv2(out)
        out = self.act(out)
        out = self.maxpool(out)

        out = self.conv3(out)
        out = self.act(out)

        out = self.adaptivepool(out)
        out = self.flatten(out)
        out = self.linear(out)
        out = self.act(out)
        out = self.linear2(out)



        return out


In [ ]:
model = ConvNet()
model

ConvNet(
  (act): LeakyReLU(negative_slope=0.2)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (adaptivepool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=128, out_features=20, bias=True)
  (linear2): Linear(in_features=20, out_features=2, bias=True)
)

In [ ]:
for sample in train_loader:

    img = sample['img']
    label = sample['label']
    model(img)
    break


In [ ]:
loss_fn = nn.CrossEntropyLoss()
opimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas = (0.9, 0.999))


In [ ]:
def accuracy(pred, label):
    answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)
    return answer.mean()

In [ ]:
epochs = 10
for epoch in range(epochs):
    loss_val = 0
    acc_val = 0
    for sample in (pbar := tqdm.tqdm(train_loader)):
        img, label = sample['img'], sample['label']
        opimizer.zero_grad()

        label = F.one_hot(label, 2).float()
        pred = model(img)

        loss = loss_fn(pred, label)

        loss.backward()
        loss_item = loss.item()
        loss_val += loss_item

        opimizer.step()
        acc_current = accuracy(pred, label)
        acc_val += acc_current

    pbar.set_description(f'loss:{loss_item:.4e}')
    print(loss_val/len(train_loader))
    print(acc_val/len(train_loader))


  0%|          | 0/500 [00:00<?, ?it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_7176\1416373963.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)
100%|██████████| 500/500 [00:46<00:00, 10.83it/s]


0.6880267058610916
0.542


100%|██████████| 500/500 [00:45<00:00, 10.90it/s]


0.662888898730278
0.611


100%|██████████| 500/500 [00:44<00:00, 11.25it/s]


...

0.398545144289732
0.818125


100%|██████████| 500/500 [00:39<00:00, 12.58it/s]


0.3725748079270124
0.824


100%|██████████| 500/500 [00:40<00:00, 12.49it/s]

0.3288432320356369
0.8515
